In [3]:
import sympy as sy
import numpy as np

In [68]:
class GenericWaveguide:
    """
    Generic properties of a waveguide
    """
    def __init__(
        self,
        critical_incident_angle = None,
        incident_angle = None,
        n_core = None,
        n_cladding = None):
        
       
        self.n_core = n_core
        self.n_cladding = n_cladding
        if(incident_angle):
            self.incident_angle = np.deg2rad(incident_angle) # unit:degrees
        else:
            self.incident_angle = incident_angle # unit:degrees
    
    @property
    def critical_incident_angle(self):
        critical_incident_angle = sy.asin(self.n_core**2 - self.n_cladding**2).evalf()
        return np.rad2deg(float(critical_incident_angle))
    
    @property
    def relative_refractive_index(self):
        return (n_core**2 - n_cladding**2)/(2 * n_cladding**2)
    
    @property
    def numerical_aperture(self):
        return n_core * sy.sqrt(2 * self.relative_refractive_index)
    
    @property
    def mode_propagation_constants(self, k_mode):
        beta = k_mode * n_core * sy.cos(self.incident_angle)
        kappa = k_mode * n_core * sy.sin(self.incident_angle)
        return {"beta": beta,
                "kappa": kappa}
    
    @property
    def reflection_cofficient(self):
        reflected_amplitude = self.n_core * sy.sin(self.incident_angle) + \
                              1j * sy.cos((self.n_core**2) * sy.cos(self.incident_angle)**2 - self.n_cladding**2)
        incident_amplitude = self.n_core * sy.sin(self.incident_angle) - \
                             1j * sy.cos((self.n_core**2) * sy.cos(self.incident_angle)**2 - self.n_cladding**2)
        return reflected_amplitude / incident_amplitude
    
    @property
    def phase_shift(self):
        return -2 * sy.atan(sy.sqrt(self.n_core**2 * sy.cos(self.incident_angle)**2 - self.n_cladding**2) / \
                           (self.n_core * sy.sin(self.incident_angle)))

In [69]:
a = GenericWaveguide(n_core = 1, n_cladding = 1.1, incident_angle=10)
a.critical_incident_angle

-12.122352244789123

In [99]:
a.reflection_cofficient.evalf()

-0.938056072083378 + 0.34648348536042*I

In [128]:
DEFAULT_SIMULATION_CONFIGURATION = {
    "start_time": 0,
    "duration": 1e-6,
    "data_points": 100
}

DEFAULT_WAVE_PROPERTIES = {
    "ideal_wavelength" : 700e-9,
}

DEFAULT_PROPAGATION_CONSTANTS = {"beta": None,
                                 "kappa": None}

class Light:
    """
    Generic instance with all properties of inputted, incident light.
    """
    def __init__(self):
        pass
    
    @property
    def simulation_configuration(self,
                                 simulation_configuration = DEFAULT_SIMULATION_CONFIGURATION):
        simulation_time = np.linspace(simulation_configuration.start_time,
                                      simulation_configuration.duration,
                                      simulation_configuration.data_points)
        return simulation_time
    
    @property
    def wave_configuration(self,
                           wave_configuration = DEFAULT_WAVE_PROPERTIES,
                           propagation_constants = DEFAULT_PROPAGATION_CONSTANTS,
                          ):
        configuration = {}
        configuration.update(wave_configuration)
        configuration.update(propagation_constants)
        configuration["ideal_angular_velocity"] = configuration["ideal_wavelength"] * 2 * sy.pi
        return configuration
    
    @classmethod
    def electric_field_vector(self):
        pass

In [126]:
l = Light()
l.wave_configuration

{'ideal_wavelength': 7e-07,
 'beta': None,
 'kappa': None,
 'ideal_angular_velocity': 1.4e-6*pi}

In [127]:
l.electric_field_vector()

In [ ]:
class SlabWaveguide(GenericWaveguide):
    